# Molecular Simulation 
Carrying out molecular dynamics (MD) and Monte-Carlo (MC) simulations using a (trained) potential. 

In [1]:
from utils import set_env
# set_env('.env')

In [2]:
import os
os.environ["JAX_ENABLE_X64"] = "1"
os.environ["JAX_PLATFORM_NAME"] = "cpu" 
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false" 

In [3]:
from pantea.datasets import RunnerDataset
from pantea.simulation import (
    MDSimulator, 
    BrendsenThermostat, 
    MCSimulator,
    run_simulation
)
from pantea.atoms import Structure
from pantea.units import units as units

import matplotlib.pylab as plt
from pathlib import Path
from ase import Atoms
from ase.visualize import view
import ase.io
import jax.numpy as jnp

In [4]:
base_dir = Path('LJ')

## Data

In [6]:
d = 6  # Angstrom
uc = Atoms('He', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
s0 = Structure.from_ase(uc.repeat((7, 7, 7)))

# d = 10  # Angstrom
# uc = Atoms('Ar', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
# s0 = Structure.create_from_ase(uc.repeat((7, 7, 7)))

atoms = s0.to_ase()
# view(atoms, viewer='x3d') # ase, ngl

## Potential

In [8]:
from pantea.types import Array
import jax
from functools import partial
from pantea.simulation.lj import LJPotential

# He
ljpot = LJPotential(
    sigma=2.5238 * units.FROM_ANGSTROM,  # Bohr
    epsilon=4.7093e-04 * units.FROM_ELECTRON_VOLT,  # Hartree
    r_cutoff=6.3095 * units.FROM_ANGSTROM,  # 2.5 * sigma
)

# Ar
# ljpot = LJPotential(
#     sigma=3.405 * units.FROM_ANGSTROM,                       # Bohr
#     epsilon=0.01032439284 * units.FROM_ELECTRON_VOLT,        # Hartree
#     r_cutoff=8.5125 * units.FROM_ANGSTROM,                   # 2.5 * sigma
# )

## Molecular Dynamics (MD)

In [9]:
# v0 = MDSimulator.generate_random_velocity(temperature=300.0, mass=s0.mass, seed=2023)
# brendsen = BrendsenThermostat(target_temperature=300.0, time_constant=50.0 * units.FROM_FEMTO_SECOND)

md = MDSimulator(
    potential=ljpot,
    initial_structure=s0,
    time_step=0.5 * units.FROM_FEMTO_SECOND,
    temperature=300, # K
    # initial_velocity=v0,
    # thermostat=brendsen
)

In [10]:
md.get_pressure()

Array(6.51659916e-07, dtype=float64)

In [11]:
md.get_total_energy()

Array(0.48832374, dtype=float64)

In [12]:
# Warmp up
# run_simulation(md)

# %timeit run_simulation(md, num_steps=1, output_freq=-1)

In [13]:
run_simulation(md, num_steps=1000, output_freq=100, filename="md.xyz")

0          time[ps]:0.00000    Temp[K]:299.95012  Etot[Ha]:0.4883237353    Epot[Ha]:-0.0003923654   Pres[kb]:0.19173   
100        time[ps]:0.05000    Temp[K]:180.48925  Etot[Ha]:0.2937277968    Epot[Ha]:-0.0003477770   Pres[kb]:0.11535   
200        time[ps]:0.10000    Temp[K]:340.79198  Etot[Ha]:0.5549463578    Epot[Ha]:-0.0003143736   Pres[kb]:0.21783   
300        time[ps]:0.15000    Temp[K]:332.48107  Etot[Ha]:0.5414355956    Epot[Ha]:-0.0002839803   Pres[kb]:0.21252   
400        time[ps]:0.20000    Temp[K]:339.34123  Etot[Ha]:0.5526201298    Epot[Ha]:-0.0002768727   Pres[kb]:0.21688   
500        time[ps]:0.25000    Temp[K]:325.52335  Etot[Ha]:0.5301128543    Epot[Ha]:-0.0002703262   Pres[kb]:0.20803   
600        time[ps]:0.30000    Temp[K]:323.53786  Etot[Ha]:0.5268878850    Epot[Ha]:-0.0002602887   Pres[kb]:0.20678   
700        time[ps]:0.35000    Temp[K]:298.66455  Etot[Ha]:0.4863645128    Epot[Ha]:-0.0002569646   Pres[kb]:0.19088   
800        time[ps]:0.40000    Temp[K]:3

In [14]:
md.get_pressure() * units.TO_KILO_BAR

Array(0.20212924, dtype=float64)

In [15]:
md.positions * units.TO_ANGSTROM

Array([[ 1.11870489,  4.24844493,  2.75083361],
       [ 0.56430649,  2.14820432,  8.35313146],
       [ 3.85854755,  3.8140108 , 13.61781432],
       ...,
       [36.87880675, 37.60983005, 23.85054836],
       [37.4298655 , 40.55351055, 30.59379365],
       [37.90171642, 35.95951402, 38.96970769]], dtype=float64)

In [17]:
atoms = md.get_structure().to_ase()
# view(atoms, viewer='ngl') # ase, ngl

## Monte Carlo (MC)

In [18]:
mc = MCSimulator(
    potential=ljpot,
    initial_structure=s0,
    temperature=300, # K
    translate_step=0.3 * units.FROM_ANGSTROM,
    movements_per_step=10,
)

In [19]:
# Warmp up
# run_simulation(mc)

# %timeit run_simulation(mc, num_steps=1, output_freq=-1)

In [21]:
run_simulation(mc, num_steps=1000, output_freq=100, filename=Path("mc.xyz"))

1000       Temp[K]:300.0000000000  Epot[Ha]:-0.0008474013   
1100       Temp[K]:300.0000000000  Epot[Ha]:-0.0005124811   
1200       Temp[K]:300.0000000000  Epot[Ha]:-0.0006410031   
1300       Temp[K]:300.0000000000  Epot[Ha]:-0.0008401709   
1400       Temp[K]:300.0000000000  Epot[Ha]:0.0001769136    
1500       Temp[K]:300.0000000000  Epot[Ha]:0.0002753122    
1600       Temp[K]:300.0000000000  Epot[Ha]:-0.0004646807   
1700       Temp[K]:300.0000000000  Epot[Ha]:0.0003113115    
1800       Temp[K]:300.0000000000  Epot[Ha]:0.0012666432    
1900       Temp[K]:300.0000000000  Epot[Ha]:0.0024171232    
2000       Temp[K]:300.0000000000  Epot[Ha]:0.0002391148    


In [22]:
mc.positions * units.TO_ANGSTROM

Array([[ 4.515164 ,  3.3334563,  4.6523848],
       [ 4.818183 ,  2.2647018, 11.128823 ],
       [ 4.290415 ,  2.333686 , 14.79884  ],
       ...,
       [39.093845 , 39.597973 , 26.38749  ],
       [37.077824 , 36.643116 , 33.387676 ],
       [40.69105  , 36.773254 , 40.958164 ]], dtype=float32)

In [23]:
atoms = mc.get_structure().to_ase()
# view(atoms, viewer='ngl') # ase, ngl